In [1]:
import os
from openai import OpenAI
import ollama
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
import gradio as gr

In [2]:
load_dotenv()

openapi_key = os.getenv("OPENAI_API_KEY")

openai = OpenAI()

system_prompt = "You are an assistant who is expert in technology and coding, specially python and llms. You need to behave as a technical tutor and answer technical questions in Markdown"

def get_user_prompt(ques):
    prompt = ques
    return prompt

In [11]:
def ask_gpt_tutor(ques):
    MODEL = 'gpt-4o-mini'
    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": get_user_prompt(ques)}]
    stream = openai.chat.completions.create(model = MODEL, messages = messages, stream = True)
    #display(Markdown(response.choices[0].message.content))
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield response

In [15]:
def ask_ollama_tutor(question):
    MODEL = "llama3.2:1b"
    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": get_user_prompt(question)}]
    response = ollama.chat(model = MODEL, messages = messages, stream=True)
    result = ""
    for chunk in response:
        result += chunk["message"]["content"]
        yield result

In [18]:
def stream_model(prompt, model):
    if model=="GPT":
        result = ask_gpt_tutor(prompt)
    elif model=="Ollama":
        result = ask_ollama_tutor(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result
        

In [19]:
view = gr.Interface(
    fn=stream_model,
    inputs=[
        gr.Textbox(label="Ask me any technical question"),
        gr.Dropdown(["GPT", "Ollama"], label="Select Model")
    ],
    outputs=gr.Markdown(label="Response"),
    allow_flagging="never"
)

view.launch()

/Users/mannatwadhwani/Learning_Workspace/my_llm_learning/llms/lib/python3.10/site-packages/gradio/interface.py:399: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.
